### Load Data

In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import pandas as pd
import numpy as np
import datetime
import os
import itertools
import pywt

In [ ]:
def findTime(x):
    return x.time()
def findDate(x):
    return x.date()

road_data = pd.read_csv('train_data/data/5684.csv')
xianquan_str='5684'
road_data['FROMTIME'] = pd.to_datetime(road_data['FROMTIME'])
road_data.index = road_data['FROMTIME']
road_data['FROMTIME-Time'] = road_data['FROMTIME'].apply(findTime)  # 找到时间
road_data['FROMTIME-Date'] = road_data['FROMTIME'].apply(findDate)  # 找到日期
road_data

plt.figure(figsize= (20,5))
plt.title(str(5684))
for date in road_data['FROMTIME-Date'].unique():
    df_temp = road_data[road_data['FROMTIME-Date'] == date]
    data3 = df_temp.resample(datetime.timedelta(seconds=5 * 60)).sum()
    data3.index = pd.Series(data3.index).apply(findTime)
    plt.plot(data3.index,data3['FLOW'],label=date)
plt.legend()

### Divide the data into weekend and weekday

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM,Bidirectional
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from keras import regularizers
from sklearn import metrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10

In [ ]:
xianquan = road_data.resample(datetime.timedelta(seconds=5 * 60)).sum()
df = xianquan.copy()
df.shape

In [ ]:
df['FROMTIME'] = pd.to_datetime(df.index)
df['FROMTIME-Time'] = df["FROMTIME"].apply(findTime)  # 找到时间
df['FROMTIME-Date'] = df["FROMTIME"].apply(findDate)  # 找到日期
df=df[(pd.to_datetime(df["FROMTIME-Time"],format = '%H:%M:%S')>= pd.to_datetime('07:00:00',format = '%H:%M:%S'))&(pd.to_datetime(df["FROMTIME-Time"],format = '%H:%M:%S')<= pd.to_datetime('22:00:00',format = '%H:%M:%S'))]

In [ ]:

weekend_list=["2019-07-06","2019-07-07","2019-07-13","2019-07-14","2019-07-20","2019-07-21","2019-07-27","2019-07-28"]

df_weekend=pd.DataFrame()
for day in weekend_list:
    temp=df[pd.to_datetime(df['FROMTIME-Date'],format = '%Y-%m-%d')==pd.to_datetime(day,format = '%Y-%m-%d')]
    df_weekend=pd.concat([df_weekend,temp])

df_weekend.index = df_weekend['FROMTIME']
df_weekend.to_csv('train_data/lstm/weekend_%s.csv'%xianquan_str)

In [ ]:
weekday_list=["2019-07-01","2019-07-02","2019-07-03","2019-07-04","2019-07-05","2019-07-08","2019-07-09","2019-07-10",
             "2019-07-11","2019-07-12","2019-07-15","2019-07-16","2019-07-17","2019-07-18","2019-07-19","2019-07-22",
             "2019-07-23","2019-07-24","2019-07-25","2019-07-26","2019-07-29","2019-07-30","2019-07-31"]
df_weekday=pd.DataFrame()
for day in weekday_list:
    temp=df[pd.to_datetime(df['FROMTIME-Date'],format = '%Y-%m-%d')==pd.to_datetime(day,format = '%Y-%m-%d')]
    df_weekday=pd.concat([df_weekday,temp])

df_weekday.index = df_weekday['FROMTIME']
df_weekday.to_csv('train_data/lstm/weekday_%s.csv'%xianquan_str)

In [ ]:
plt.figure(figsize= (20,5))
plt.title(str('%s'%xianquan_str))
# plt.plot(df_weekend.index,df_weekend['COUNT'])
# plt.plot(df_weekday.index,df_weekday['COUNT'])
plt.plot(df['COUNT'])

### Wavelet denoising decomposes data

In [ ]:
import pywt

In [ ]:
# Wavelet noise filtering
def wavelet_denoising(data):
    #  wavelet function takes db4
    db4 = pywt.Wavelet('db4')
    #  decompose
    coeffs = pywt.wavedec(data, db4)
    # high frequency coefficient is zeroed
    coeffs[len(coeffs)-1] *= 0
    coeffs[len(coeffs)-2] *= 0
    # resconstruct
    meta = pywt.waverec(coeffs, db4)
    return meta

#### Train for weekend data

In [ ]:
df_weekend=pd.read_csv('train_data/lstm/weekend_%s.csv'%xianquan_str)
df_weekday=pd.read_csv('train_data/lstm/weekday_%s.csv'%xianquan_str)

df_weekday.index=df_weekday["FROMTIME"]
df_weekend.index=df_weekend["FROMTIME"]

data1=df_weekend[(pd.to_datetime(df_weekend.index,format = '%Y-%m-%d %H:%M:%S')<= pd.to_datetime('2019-07-27 22:00:00',format = '%Y-%m-%d %H:%M:%S'))]
data2=df_weekend[(pd.to_datetime(df_weekend.index,format = '%Y-%m-%d %H:%M:%S')> pd.to_datetime('2019-07-27 22:00:00',format = '%Y-%m-%d %H:%M:%S'))]
data1.to_csv('train_data/train_%s.csv'%xianquan_str)
data2.to_csv('train_data/test_%s.csv'%xianquan_str)

data3=df_weekday[(pd.to_datetime(df_weekday.index,format = '%Y-%m-%d %H:%M:%S')<= pd.to_datetime('2019-07-30 22:00:00',format = '%Y-%m-%d %H:%M:%S'))]
data4=df_weekday[(pd.to_datetime(df_weekday.index,format = '%Y-%m-%d %H:%M:%S')> pd.to_datetime('2019-07-30 22:00:00',format = '%Y-%m-%d %H:%M:%S'))]
data3.to_csv('train_data/train_%s.csv'%xianquan_str)
data4.to_csv('train_data/test_%s.csv'%xianquan_str)

In [ ]:
lag = 12
file1 = 'train_data/train_%s.csv'%xianquan_str
file2 = 'train_data/test_%s.csv'%xianquan_str

df1 = pd.read_csv(file1, encoding='utf-8').fillna(0)
df2 = pd.read_csv(file2, encoding='utf-8').fillna(0)

In [ ]:
# Training data: data1_temp is the low frequency band of weekend data, and data1_zao is the noise segment of weekend data
data_list=np.array(df1["COUNT"])

data=[]
for i in range(len(data_list)):
    if data_list[i]>0.01:
        data.append(data_list[i])
data_list=data

data1_temp=wavelet_denoising(data_list)
data1_zao=[]
for i in range(len(data_list)):
    tmp=data_list[i]-data1_temp[i]
    data1_zao.append(tmp)
data1_zao=np.array(data1_zao)

In [ ]:
# Test data: data2_temp is the low frequency band of weekend data, and data2_zao is the noise segment of weekend data
data_list2=np.array(df2["COUNT"])
data2_temp=wavelet_denoising(data_list2)
df_data2 = np.array(df2["COUNT"])
data2_zao=[]
for i in range(len(df_data2)):
    tmp=df_data2[i]-data2_temp[i]
    data2_zao.append(tmp)
data2_zao=np.array(data2_zao)

In [ ]:
# Noise data to be trained
plt.figure(figsize= (20,5))
plt.title(str('%s'%xianquan_str))
plt.plot(data1_zao)

In [ ]:
# Low frequency data to be trained
plt.figure(figsize= (20,5))
plt.title(str('%s'%xianquan_str))
plt.plot(data1_temp)

#### Prapare training function

In [ ]:
def process_data(train, test, lags):
    scaler = MinMaxScaler(feature_range=(0, 1)).fit(train.reshape(-1, 1))
    flow1 = scaler.transform(train.reshape(-1, 1)).reshape(1, -1)[0]
    flow2 = scaler.transform(test.reshape(-1, 1)).reshape(1, -1)[0]

    train, test = [], []
    for i in range(lags, len(flow1)):
        if (i/12)%15==0:
            if i==180:
                temp=flow1[i - lags: i + 12]
                if len(temp)==24:
                    train.append(temp)
        else:
            
            temp=flow1[i - lags: i + 12]
            if len(temp)==24:
                train.append(temp)
                
    for i in range(lags, len(flow2),12):
        if (i/12)%16==0:
            pass
        else:
            temp=flow2[i - lags: i + 12]
            if len(temp)==24:
                test.append(temp)

    train = np.array(train)
    test = np.array(test)
    np.random.shuffle(train)
    X_train = train[:,0:12]
    y_train = train[:,-12:]
    X_test = test[:,0:12]
    y_test = test[:,-12:]
    
    return X_train, y_train, X_test, y_test, scaler



def process_data_zao(train, test, lags):
    scaler_zao = MinMaxScaler(feature_range=(0, 1)).fit(train.reshape(-1, 1))
    flow1 = scaler_zao.transform(train.reshape(-1, 1)).reshape(1, -1)[0]
    flow2 = scaler_zao.transform(test.reshape(-1, 1)).reshape(1, -1)[0]

    train, test = [], []
    for i in range(lags, len(flow1)):
        if (i/12)%15==0:
            if i==180:
                temp=flow1[i - lags: i + 12]
                if len(temp)==24:
                    train.append(temp)
        else:
           
            temp=flow1[i - lags: i + 12]
            if len(temp)==24:
                train.append(temp)
                
    for i in range(lags, len(flow2),12):
        if (i/12)%16==0:
            pass
        else:
            temp=flow2[i - lags: i + 12]
            if len(temp)==24:
                test.append(temp)

    train = np.array(train)
    test = np.array(test)
    np.random.shuffle(train)
    X_train = train[:,0:12]
    y_train = train[:,-12:]
    X_test = test[:,0:12]
    y_test = test[:,-12:]
    
    return X_train, y_train, X_test, y_test, scaler_zao

In [ ]:
def get_lstm(units):
    """LSTM(Long Short-Term Memory)
    Build LSTM Model.

    # Arguments
        units: List(int), number of input, output and hidden units.
    # Returns
        model: Model, nn model.
    """

    model = Sequential()
    model.add(LSTM(units[1], activation='tanh',input_shape=(units[0], 1),return_sequences=True))
    model.add(Dropout(0.25))
    model.add(LSTM(units[2],activation='tanh'))
    model.add(Dropout(0.25))
    model.add(Dense(units=units[3],activation='linear')) 

    return model

In [ ]:
"""

 Training: LSTM and GRU are trained according to the normal RNN network. Use the train_model() function to train. 
 
RMSprop(lr=0.001, rho=0.9, epsilon=1e-06) was used as the optimizer with batch_szie of 256 and lags of 12(i.e., lags of one hour).
"""
def train_model(model, X_train, y_train, name, config,wd_str,is_zao):
    """train
    train a single model.

    # Arguments
        model: Model, NN model to train.
        X_train: ndarray(number, lags), Input data for train.
        y_train: ndarray(number, ), result data for train.
        name: String, name of model.
        config: Dict, parameter for train.
    """

    model.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])#optimizer原先是rmsprop
    model.summary()
    # early = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')
    hist = model.fit(
        X_train, y_train,
        batch_size=config["batch"],
        epochs=config["epochs"],
        validation_split=0.05)# original parameter was 0.05
    if is_zao==False:
        if wd_str=='weekday':
            model.save('train_data/lstm/weekday_model/xiaobo%s.h5'%xianquan_str)
            df = pd.DataFrame.from_dict(hist.history)
            df.to_csv('train_data/lstm/weekday_model/xiaobo%sloss.csv'%xianquan_str, encoding='utf-8', index=False)
        else:
            model.save('train_data/lstm/weekend_model/xiaobo%s.h5'%xianquan_str)
            df = pd.DataFrame.from_dict(hist.history)
            df.to_csv('train_data/lstm/weekend_model/xiaobo%sloss.csv'%xianquan_str, encoding='utf-8', index=False)  
    else:
        if wd_str=='weekday':
            model.save('train_data/lstm/weekday_model/xiaobo_zao%s.h5'%xianquan_str)
            df = pd.DataFrame.from_dict(hist.history)
            df.to_csv('train_data/lstm/weekday_model/xiaobo_zao%sloss.csv'%xianquan_str, encoding='utf-8', index=False)
        else:
            model.save('train_data/lstm/weekend_model/xiaobo_zao%s.h5'%xianquan_str)
            df = pd.DataFrame.from_dict(hist.history)
            df.to_csv('train_data/lstm/weekend_model/xiaobo_zao%sloss.csv'%xianquan_str, encoding='utf-8', index=False)  

In [ ]:
def MAPE(y_true, y_pred):
    """Mean Absolute Percentage Error
    Calculate the mape.

    # Arguments
        y_true: List/ndarray, ture data.
        y_pred: List/ndarray, predicted data.
    # Returns
        mape: Double, result data for train.
    """

    y = [x for x in y_true if x > 0]
    y_pred = [y_pred[i] for i in range(len(y_true)) if y_true[i] > 0]

    num = len(y_pred)
    sums = 0

    for i in range(num):
        tmp = abs(y[i] - y_pred[i]) / y[i]
        sums += tmp
    if num!=0:
        mape = sums * (100 / num)
    else:
        mape=0
    return mape


def eva_regress(y_true, y_pred):
    """Evaluation
    evaluate the predicted resul.

    # Arguments
        y_true: List/ndarray, ture data.
        y_pred: List/ndarray, predicted data.
    """

    mape = MAPE(y_true, y_pred)
    vs = metrics.explained_variance_score(y_true, y_pred)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    print('explained_variance_score:%f' % vs)
    print('mape:%f%%' % mape)
    print('mae:%f' % mae)
    print('mse:%f' % mse)
    print('rmse:%f' % math.sqrt(mse))
    print('r2:%f' % r2)

In [ ]:
X_train, y_train, X_test, y_test, scaler= process_data(data1_temp, data2_temp, lag)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

#### Low frequency data training

In [ ]:
wd_str='weekend'
is_zao=False

units=[X_train.shape[1],64,128,y_train.shape[1]]
model=get_lstm(units)
name='lstm'
config={"batch":64,"epochs":500}
train_model(model, X_train, y_train, name, config,wd_str,is_zao)

In [ ]:
xiaobolstm = load_model('train_data/lstm/weekend_model/xiaobo%s.h5'%xianquan_str)

model=xiaobolstm
name='xiaoboLSTM'

X_train, y_train, X_test, y_test, scaler= process_data(data1_temp, data2_temp, lag)

y_test=y_test.reshape(y_test.shape[0],y_test.shape[1])
y_test = scaler.inverse_transform(y_test)

y_preds = []
train_preds=[]

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
file = 'train_data/model' + name + '.png'
plot_model(model, to_file=file, show_shapes=True)
predicted = model.predict(X_test)

predicted = scaler.inverse_transform(predicted)

# y_preds.append(predicted)#[:12]
y_preds=predicted
print(name)

In [ ]:
d = '2019-07-08 08:00'
x = pd.date_range(d, periods=168, freq='5min')

fig = plt.figure()
ax = fig.add_subplot(111)

y_test_temp=[]
for i in range(len(y_test)):
    for j in range(len(y_test[i])):
        y_test_temp.append(y_test[i][j])

# y_test_true=[]
# for i in range(len(y_test)):
#     for j in range(len(y_test[i])):
#         tmp=y_test[i][j]+y_test_zao[i][j]
#         y_test_true.append(tmp)
    
ax.plot(x, y_test_temp, label='True Data')
# ax.plot(x, y_test_true, label='True Data')

y_preds_temp=[]
for i in range(len(y_preds)):
    for j in range(len(y_preds[i])):
        y_preds_temp.append(y_preds[i][j])
        
ax.plot(x, y_preds_temp, label='preds Data')


eva_regress(y_test_temp, y_preds_temp)
# eva_regress(y_test_true, y_preds_temp)

plt.legend()
plt.grid(True)
plt.xlabel('Time of Day')
plt.ylabel('Flow')

plt.show()

In [ ]:
X_train_zao, y_train_zao, X_test_zao, y_test_zao, scaler_zao= process_data_zao(data1_zao, data2_zao, lag)
X_train_zao = np.reshape(X_train_zao, (X_train_zao.shape[0], X_train_zao.shape[1], 1))

d = '2019-07-08 08:00'
x = pd.date_range(d, periods=168, freq='5min')

y_test_zao=y_test_zao.reshape(y_test_zao.shape[0],y_test_zao.shape[1])
y_test_zao = scaler_zao.inverse_transform(y_test_zao)

fig = plt.figure()
ax = fig.add_subplot(111)

y_test_temp=[]
for i in range(len(y_test)):
    for j in range(len(y_test[i])):
        y_test_temp.append(y_test[i][j])


y_test_true=[]
for i in range(len(y_test)):
    for j in range(len(y_test[i])):
        tmp=y_test[i][j]+y_test_zao[i][j]
        y_test_true.append(tmp)
    
# ax.plot(x, y_test_temp, label='True Data')
ax.plot(x, y_test_true, label='True Data')

y_preds_temp=[]
for i in range(len(y_preds)):
    for j in range(len(y_preds[i])):
        y_preds_temp.append(y_preds[i][j])
        
y_preds_true=[]
for i in range(len(y_preds)):
    for j in range(len(y_preds[i])):
        tmp=y_preds[i][j]+y_preds_zao[i][j]
        y_preds_true.append(tmp)

ax.plot(x, y_preds_true, label='preds Data')


# eva_regress(y_test_temp, y_preds_temp)
eva_regress(y_test_true, y_preds_true)

plt.legend()
plt.grid(True)
plt.xlabel('Time of Day')
plt.ylabel('Flow')

plt.show()

#### Add attention mechanism to train low frequency data

In [ ]:
import keras.backend as K
import numpy as np
from keras.layers import Input, Dense, LSTM, merge ,Conv1D,Dropout,Bidirectional,Multiply
from keras.models import Model
from keras.layers import merge
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
import  pandas as pd

In [ ]:
def get_activations(model, inputs, print_shape_only=False, layer_name=None):
    print('----- activations -----')
    activations = []
    inp = model.input
    if layer_name is None:
        outputs = [layer.output for layer in model.layers]
    else:
        outputs = [layer.output for layer in model.layers if layer.name == layer_name]  # all layer outputs
    funcs = [K.function([inp] + [K.learning_phase()], [out]) for out in outputs]  # evaluation functions
    layer_outputs = [func([inputs, 1.])[0] for func in funcs]
    for layer_activations in layer_outputs:
        activations.append(layer_activations)
        if print_shape_only:
            print(layer_activations.shape)
        else:
            print('shape为',layer_activations.shape)
            print(layer_activations)
    return activations

In [ ]:
def attention_3d_block2(inputs, single_attention_vector=False):
    # inputs.shape = (batch_size, time_steps, input_dim)
    time_steps = K.int_shape(inputs)[1]
    input_dim = K.int_shape(inputs)[2]
    a = Permute((2, 1))(inputs)
    a = Dense(time_steps, activation='softmax')(a)
    if single_attention_vector:
        a = Lambda(lambda x: K.mean(x, axis=1))(a)
        a = RepeatVector(input_dim)(a)

    a_probs = Permute((2, 1))(a)

    # element-wise
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

def attention_model():
    inputs = Input(shape=(TIME_STEPS, INPUT_DIMS))

    x = Conv1D(filters = 64, kernel_size = 1, activation = 'relu')(inputs)  #, padding = 'same'
    x = Dropout(0.3)(x)

    #lstm_out = Bidirectional(LSTM(lstm_units, activation='relu'), name='bilstm')(x)
    lstm_out = Bidirectional(LSTM(lstm_units, return_sequences=True))(x)
    lstm_out = Dropout(0.3)(lstm_out)
    attention_mul = attention_3d_block2(lstm_out,True)
    attention_mul = Flatten()(attention_mul)

    output = Dense(12, activation='sigmoid')(attention_mul)
    model = Model(inputs=[inputs], outputs=output)
    return model

INPUT_DIMS = 1
TIME_STEPS = 12
lstm_units = 64
wd_str='weekend'
m = attention_model()
m.summary()
m.compile(optimizer='rmsprop', loss='mse')

X_train, y_train, X_test, y_test, scaler= process_data(data1_temp, data2_temp, lag)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

m.fit([X_train], y_train, epochs=1000, batch_size=32, validation_split=0.1)

if wd_str=='weekday':
    m.save('train_data/lstm/weekday_model/xiaobo_attention_%s.h5'%xianquan_str)
else:
    m.save('train_data/lstm/weekend_model/xiaobo_attention_%s.h5'%xianquan_str) 

In [ ]:
xiaobolstm = load_model('train_data/lstm/weekend_model/xiaobo_attention_%s.h5'%xianquan_str)

model=xiaobolstm
name='xiaobo_attentionLSTM'

X_train, y_train, X_test, y_test, scaler= process_data(data1_temp, data2_temp, lag)

y_test=y_test.reshape(y_test.shape[0],y_test.shape[1])
y_test = scaler.inverse_transform(y_test)

y_preds = []
train_preds=[]

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
file = 'train_data/model' + name + '.png'
plot_model(model, to_file=file, show_shapes=True)
predicted = model.predict(X_test)
train_predicted=model.predict(X_train)
predicted = scaler.inverse_transform(predicted)
train_predicted = scaler.inverse_transform(train_predicted)
# y_preds.append(predicted)#[:12]
y_preds=predicted
print(name)

In [ ]:
d = '2019-07-08 08:00'
x = pd.date_range(d, periods=168, freq='5min')

fig = plt.figure()
ax = fig.add_subplot(111)

y_test_temp=[]
for i in range(len(y_test)):
    for j in range(len(y_test[i])):
        y_test_temp.append(y_test[i][j])

# y_test_true=[]
# for i in range(len(y_test)):
#     for j in range(len(y_test[i])):
#         tmp=y_test[i][j]+y_test_zao[i][j]
#         y_test_true.append(tmp)
    
ax.plot(x, y_test_temp, label='True Data')
# ax.plot(x, y_test_true, label='True Data')

y_preds_temp=[]
for i in range(len(y_preds)):
    for j in range(len(y_preds[i])):
        y_preds_temp.append(y_preds[i][j])
        
ax.plot(x, y_preds_temp, label='preds Data')


eva_regress(y_test_temp, y_preds_temp)
# eva_regress(y_test_true, y_preds_temp)

plt.legend()
plt.grid(True)
plt.xlabel('Time of Day')
plt.ylabel('Flow')

plt.show()

#### Noise data training

In [ ]:
X_train_zao, y_train_zao, X_test_zao, y_test_zao, scaler_zao= process_data_zao(data1_zao, data2_zao, lag)
X_train_zao = np.reshape(X_train_zao, (X_train_zao.shape[0], X_train_zao.shape[1], 1))

In [ ]:
wd_str='weekend'
is_zao=True

units=[X_train_zao.shape[1],64,64,y_train_zao.shape[1]]
model=get_lstm(units)
name='xiaobo_lstm'
config={"batch":64,"epochs":2000}
train_model(model, X_train_zao, y_train_zao, name, config,wd_str,is_zao)

In [ ]:
xiaobolstm_zao = load_model('train_data/lstm/weekend_model/xiaobo_zao%s.h5'%xianquan_str)

model=xiaobolstm_zao
name='xiaoboLSTM_zao'

X_train_zao, y_train_zao, X_test_zao, y_test_zao, scaler_zao=process_data_zao(data1_zao, data2_zao, lag)

y_test_zao=y_test_zao.reshape(y_test_zao.shape[0],y_test_zao.shape[1])
y_test_zao = scaler_zao.inverse_transform(y_test_zao)

y_preds = []
train_preds=[]

X_test_zao = np.reshape(X_test_zao, (X_test_zao.shape[0], X_test_zao.shape[1], 1))

file = 'train_data/model' + name + '.png'
plot_model(model, to_file=file, show_shapes=True)
predicted = model.predict(X_test_zao)
predicted = scaler_zao.inverse_transform(predicted)
# y_preds.append(predicted)#[:12]
y_preds_zao=predicted

print(name)

In [ ]:
for i in range(len(y_test_zao)):
    plt.figure()
    plt.plot(y_test_zao[i])
    plt.plot(y_preds_zao[i])
    eva_regress(y_test_zao[i], y_preds_zao[i])

In [ ]:
import sys 

import importlib
importlib.reload(sys)


def process_data2(train, test, lags):
    attr = 'COUNT'
    df1 = pd.read_csv(train, encoding='utf-8').fillna(0)
    df2 = pd.read_csv(test, encoding='utf-8').fillna(0)

    scaler = MinMaxScaler(feature_range=(0, 1)).fit(df1[attr].values.reshape(-1, 1))
    flow1 = scaler.transform(df1[attr].values.reshape(-1, 1))
    flow2 = scaler.transform(df2[attr].values.reshape(-1, 1))

    train, test = [], []

    for i in range(lags, len(flow1)):
        if (i/12)%15==0:
            if i==180:
                temp=flow1[i - lags: i + 12]
                if len(temp)==24:
                    train.append(temp)
        else:
            temp=flow1[i - lags: i + 12]
            if len(temp)==24:
                train.append(temp)
                
    for i in range(lags, len(flow2),12):
        if (i/12)%16==0:
            pass
        else:
            temp=flow2[i - lags: i + 12]
            if len(temp)==24:
                test.append(temp)

    train = np.array(train)
    test = np.array(test)
    np.random.shuffle(train)
    X_train = train[:,0:12]
    y_train = train[:,-12:]
    X_test = test[:,0:12]
    y_test = test[:,-12:]
    
    return X_train, y_train, X_test, y_test, scaler

In [ ]:
lstm = load_model('train_data/lstm/weekend_model/xiaobo%s.h5'%xianquan_str)

model=lstm
name='LSTM'

lag = 12
file1 = 'train_data/train_%s.csv'%xianquan_str
file2 = 'train_data/test_%s.csv'%xianquan_str
_, _, X_test, y_test, scaler = process_data(file1, file2, lag)

y_test=y_test.reshape(y_test.shape[0],y_test.shape[1])
y_test = scaler.inverse_transform(y_test)

y_preds = []
train_preds=[]

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
file = 'train_data/model' + name + '.png'
plot_model(model, to_file=file, show_shapes=True)
predicted = model.predict(X_test)
train_predicted=model.predict(X_train)
predicted = scaler.inverse_transform(predicted)
train_predicted = scaler.inverse_transform(train_predicted)
# y_preds.append(predicted)#[:12]
y_preds=predicted

print(name)

In [ ]:
for i in range(len(y_test)):
    plt.figure()
    plt.plot(y_test[i])
    plt.plot(y_preds[i])
    eva_regress(y_test[i], y_preds[i])

In [ ]:

wd_str='weekend'

units=[X_train.shape[1],64,128,y_train.shape[1]]
model=get_lstm(units)
name='lstm'
config={"batch":200,"epochs":2000}
train_model(model, X_train, y_train, name, config,wd_str)